In [2]:
import numpy as np 
import pandas as pd 
import os
from skimage.io import imread 
from PIL import Image 
from glob import glob
from sklearn.externals import joblib
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array


from keras.models import Sequential
from keras import optimizers, losses, activations, models
from keras.layers import Conv2D, Activation, Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization, GlobalMaxPool2D, Concatenate


In [ ]:
train_files = pd.DataFrame({'col':glob('train/*/*')})
test_path = pd.DataFrame({'col':glob('test/*')})
n_train = len(train_files)

In [ ]:
batch_size = 1

train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        'train/',  # this is the target directory
        target_size=(255, 255),  # all images will be resized to 255x1255
        batch_size=batch_size,
        class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels
X_train =[]
Y_train =[]
for i in range(n_train):
    x, y = train_generator.next()
    X_train.append(x[0])
    Y_train.append(y[0])
    
X_train = np.asarray(X_train)
Y_train = np.asarray(Y_train)    


In [ ]:
X_train.shape[0]

In [4]:
X_tr_filename = 'X_train_matrix.sav'
Y_tr_filename = 'Y_train_matrix.sav'
#joblib.dump(X_train, X_tr_filename)
#joblib.dump(Y_train, Y_tr_filename)

In [5]:
X_train = joblib.load(X_tr_filename)
Y_train = joblib.load(Y_tr_filename)

In [6]:
input_shape = (255, 255, 3)
nclasses =Y_train.shape[1]

In [7]:
# from sklearn.model_selection import StratifiedShuffleSplit
# sss = StratifiedShuffleSplit(n_splits=2, test_size=0.8, random_state=0)
# sss.get_n_splits(X_train, Y_train)
# for train_index, test_index in sss.split(X_train, Y_train):
#     Y_train, y_test = Y_train[train_index], Y_train[test_index]
#     X_train, x_test = X_train[train_index], X_train[test_index]
    
from sklearn.model_selection import train_test_split
n_samples=X_train.shape[0]
indices = np.arange(n_samples)
x_train, x_test, y_train, y_test, idx1, idx2 = train_test_split(X_train, Y_train, indices, test_size=0.3)

In [ ]:
del X_train, Y_train

In [8]:

def cnn_model1():
     
    inp = Input(shape=input_shape)
    norm_inp = BatchNormalization()(inp)
    X = Convolution2D(16, kernel_size=3, activation=activations.relu, padding="same")(norm_inp)
    X = Convolution2D(16, kernel_size=3, activation=activations.relu, padding="same")(X)
    X = MaxPooling2D(pool_size=(3, 3))(X)
    X = Dropout(rate=0.2)(X)
    
    X = Convolution2D(32, kernel_size=3, activation=activations.relu, padding="same")(X)
    X = Convolution2D(32, kernel_size=3, activation=activations.relu, padding="same")(X)
    X = MaxPooling2D(pool_size=(3,3))(X)
    X = Dropout(rate=0.2)(X)
    
    X = Convolution2D(64, kernel_size=2, activation=activations.relu, padding="same")(X)
    X = Convolution2D(20, kernel_size=2, activation=activations.relu, padding="same")(X)
    X = GlobalMaxPool2D()(X)
    X = Dropout(rate=0.2)(X)
    
    dense_1 = Dense(nclass, activation=activations.softmax)(dense_1)

    model = models.Model(inputs=inp, outputs=dense_1)
    opt = optimizers.Adam()
    
    model.compile(optimizer=opt, loss=losses.categorical_crossentropy, metrics=['acc'])
    model.summary()
    
    return model

In [9]:

def cnn_model2():
    Input_shape = (255, 255, 3)
    nclasses =10

    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=Input_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nclasses))
    model.add(Activation('sigmoid'))

    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    return model

In [10]:
def cnn_model3():
    input_shape = (255, 255, 3)
    nclasses =10
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=input_shape))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
 
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
 
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
 
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nclasses, activation='softmax'))
    
    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    
    return model

In [11]:
model_2 = cnn_model2()
history = model_2.fit(x_train, y_train, validation_split=0.1, epochs=10, shuffle=True, verbose=2)

Train on 1732 samples, validate on 193 samples
Epoch 1/10
 - 86s - loss: 2.3429 - acc: 0.1386 - val_loss: 2.2203 - val_acc: 0.2435
Epoch 2/10
 - 82s - loss: 2.1604 - acc: 0.2188 - val_loss: 2.0499 - val_acc: 0.2332
Epoch 3/10
 - 81s - loss: 1.9990 - acc: 0.2598 - val_loss: 1.9054 - val_acc: 0.3057
Epoch 4/10
 - 81s - loss: 1.8652 - acc: 0.3274 - val_loss: 2.6021 - val_acc: 0.1969
Epoch 5/10
 - 82s - loss: 1.6888 - acc: 0.3799 - val_loss: 1.6987 - val_acc: 0.3472
Epoch 6/10
 - 85s - loss: 1.5808 - acc: 0.4301 - val_loss: 2.2764 - val_acc: 0.3109
Epoch 7/10
 - 83s - loss: 1.4046 - acc: 0.5144 - val_loss: 2.2639 - val_acc: 0.3420
Epoch 8/10
 - 96s - loss: 1.2175 - acc: 0.5704 - val_loss: 1.9230 - val_acc: 0.4301
Epoch 9/10
 - 83s - loss: 1.0419 - acc: 0.6334 - val_loss: 2.2715 - val_acc: 0.2850
Epoch 10/10
 - 90s - loss: 0.9060 - acc: 0.6980 - val_loss: 1.9943 - val_acc: 0.4301


In [14]:
#model_2.save_weights('cnn_model2.h5')
#y_predicted = model_2.predict(x_test)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_predicted, labels=None, sample_weight=None)

In [ ]:
model_3 = cnn_model3()
history = model.fit(x_train, y_train, validation_split=0.1, epochs=2, shuffle=True, verbose=2)

In [17]:
scores = model_2.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

825/825 [==============================] - 12s 15ms/step
('Test loss:', 2.1582406971671366)
('Test accuracy:', 0.4327272726550247)
